# Packages

In [1]:
import os

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Data preprocessing

# 2020 Summer

## Plant growth factors

In [2]:
pg_df = []

In [3]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_S/node_length.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_len'])

pg_df.append(temp_df)

In [4]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_S/node_diameter.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_dia'])

pg_df.append(temp_df)

In [5]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_S/plant_height.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['plant_h'])

pg_df.append(temp_df)

In [6]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_S/leaf_area.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])/10000
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['leaf_area'])

pg_df.append(temp_df)

In [7]:
pg_df = pd.concat(pg_df, axis=1)
pg_df['DAT'] = [27]*3 + [55]*3 + [86]*3 + [112]*4 + [128]*20
pg_df['date'] = pg_df.index
pg_df = pg_df.set_index(['DAT'])

In [8]:
pg_reg_df = []
plot_x = [_ for _ in range(14, 129)]
date_x = pd.date_range('2020-03-11', '2020-07-03', freq='1d')

In [9]:
pg_df.iloc[-2, 0] = np.nan
pg_df = pg_df.dropna()

In [10]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [11]:
x = pg_df.index
y = pg_df['node_len']*10
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/10)

0.7337787624866217


In [12]:
x = pg_df.index
y = pg_df['node_dia']*10
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/10)

0.7443034188067146


In [13]:
x = pg_df.index
y = pg_df['plant_h']
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y)

0.9575512554927045


In [14]:
x = pg_df.index
y = pg_df['leaf_area']*1000
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/1000)

0.868271888891615


In [15]:
pg_reg_df = pd.DataFrame(pg_reg_df).T
pg_reg_df.columns = ['node_len', 'node_dia', 'plant_h', 'leaf_area']
pg_reg_df.index = date_x

## Fresh weight (device)

In [16]:
SW2_df = pd.read_csv('../ch2_weighing/results/2020_S/SW2_greenhouse.csv', index_col='Unnamed: 0')
SW2_df.index = pd.DatetimeIndex(SW2_df.index)

SW2_df = SW2_df.loc['2020-03-05 00:00:00': '2020-07-03 23:59:00']
SW2_df = SW2_df.interpolate()

In [17]:
rockwool_slab = np.array([626, 708, 650, 642])
rockwool_cube = np.array([48, 46, 50, 48, 46])
rockwool = (rockwool_slab + rockwool_cube.mean()*4)/1000
rockwool_mean = rockwool.mean()

In [18]:
substrate_volume = (120*12*7.5 + 10*10*6.5*4)/1000
water_w_df = substrate_volume*SW2_df['subs_VWC']/100
SW2_df['water'] = water_w_df

In [19]:
SW2_df.loc[:, 'loadcell_1'] = SW2_df.loc[:, 'loadcell_1'] - rockwool_mean
SW2_df.loc[:, 'loadcell_2'] = SW2_df.loc[:, 'loadcell_2'] - rockwool_mean
SW2_df.loc[:, 'loadcell_3'] = SW2_df.loc[:, 'loadcell_3'] - rockwool_mean

In [20]:
weight_df = pd.read_csv('../ch2_weighing/results/2020_S/weight.csv', index_col='Unnamed: 0')
weight_df.index = pd.DatetimeIndex(weight_df.index)
weight_df.index = np.append(weight_df.index[:-20], pd.DatetimeIndex(['2020-07-03']*20))
wweight_df = weight_df[['Stem FW', 'Leaf FW', 'petiole FW', 'Idv fruit FW']].sum(axis=1)

In [21]:
roots_DW = np.array([301.25, 293.55, 315.15, 356.45, 341.55, 267.65, 303.55, 231.05, 253.05, 272.35, 334.35])/1000
roots_DW_mean = roots_DW.mean()
rs_ratio_df = pd.DataFrame([0.23255813953488372, 0.14789272030651343, 0.11954022988505748, 0.13678160919540233, 0.2835814088817321], index=weight_df.index.unique(), columns=['RS_ratio'])
DW_sum_df = weight_df[[col for col in weight_df.columns if col.endswith('DW')]].sum(axis=1).groupby(weight_df.index).mean()

In [22]:
roots_df = (DW_sum_df.T * rs_ratio_df.T).T
roots_df.columns = ['root DW']
roots_df['root FW'] = roots_df['root DW']/0.1325
roots_df.index = pd.DatetimeIndex(roots_df.index)
wweight_wr_df = wweight_df.add(roots_df['root FW'])

In [23]:
night_df = SW2_df.loc[SW2_df['rad'] <= 0.2, 'loadcell_1':'loadcell_3']

In [24]:
device_df = pd.concat([(night_df.resample('1d').mean()['loadcell_1'] - SW2_df['water'].resample('1d').mean())/4,
                       (night_df.resample('1d').mean()['loadcell_2'] - SW2_df['water'].resample('1d').mean())/4,
                       (night_df.resample('1d').mean()['loadcell_3'] - SW2_df['water'].resample('1d').mean())/4], axis=1)
device_df.columns = ['CT_1', 'CT_2', 'CT_3']

## Fresh & dry weight (destructive investigation)

In [25]:
desinv_df = pd.read_csv('../ch2_weighing/results/2020_S/weight_ct.csv', index_col='Unnamed: 0')
desinv_df.index = pd.DatetimeIndex(desinv_df.index)
desinv_df.index = np.append(desinv_df.index[:-20], pd.DatetimeIndex(['2020-07-03']*20))

In [26]:
harvest_df = pd.read_excel('./data/harvest_FWnDW_2020S.xlsx', index_col='date', sheet_name='A2')
harvest_df.index = pd.DatetimeIndex(harvest_df.index)
harvest_df = harvest_df.sort_index()
harvest_df = harvest_df.groupby(harvest_df.index).sum()
harvest_df.columns = ['harvest_fw', 'harvest_dw']

In [27]:
dw_df = desinv_df.loc[:, ['Stem DW', 'Leaf DW', 'petiole DW', 'Fruit DW']]
dw_df.columns = ['stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw']
fw_df = desinv_df.loc[:, ['Stem FW', 'Leaf FW', 'petiole FW', 'Idv fruit FW']]
fw_df.columns = ['stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']

In [28]:
fw_ratio_df = fw_df.dropna().values / fw_df.dropna().sum(axis=1).values.reshape(-1, 1)
# dw_ratio_df = dw_df.dropna().values / dw_df.dropna().sum(axis=1).values.reshape(-1, 1)

In [29]:
fw_ratio_df = pd.DataFrame(fw_ratio_df, index=fw_df.dropna().index, columns=fw_df.columns)

In [30]:
fw_df = fw_df.groupby(fw_df.index).mean()
fw_df = fw_df.reindex(pg_reg_df.index)
fw_df = fw_df.interpolate().bfill()
dw_df = dw_df.groupby(dw_df.index).mean()
dw_df = dw_df.reindex(pg_reg_df.index)
dw_df = dw_df.interpolate().bfill()

In [31]:
fw_ratio_df = fw_ratio_df.groupby(fw_ratio_df.index).mean().reindex(pg_reg_df.index)
fw_ratio_df = fw_ratio_df.interpolate().bfill()

In [32]:
pg_df = pd.concat([device_df, fw_ratio_df, dw_df, pg_reg_df, harvest_df], axis=1).loc[pg_reg_df.index[0]:pg_reg_df.index[-1]]
pg_df.loc[pd.isna(pg_df['harvest_fw']), 'harvest_fw'] = 0
pg_df.loc[pd.isna(pg_df['harvest_dw']), 'harvest_dw'] = 0

In [33]:
CT1_part_df = pd.concat([pg_df['CT_1']*pg_df['stem_fw'],
                         pg_df['CT_1']*pg_df['leaf_fw'],
                         pg_df['CT_1']*pg_df['petiole_fw'],
                         pg_df['CT_1']*pg_df['fruit_fw']], axis=1)
CT2_part_df = pd.concat([pg_df['CT_2']*pg_df['stem_fw'],
                         pg_df['CT_2']*pg_df['leaf_fw'],
                         pg_df['CT_2']*pg_df['petiole_fw'],
                         pg_df['CT_2']*pg_df['fruit_fw']], axis=1)
CT3_part_df = pd.concat([pg_df['CT_3']*pg_df['stem_fw'],
                         pg_df['CT_3']*pg_df['leaf_fw'],
                         pg_df['CT_3']*pg_df['petiole_fw'],
                         pg_df['CT_3']*pg_df['fruit_fw']], axis=1)
CT1_part_df.columns = ['stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']
CT2_part_df.columns = ['stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']
CT3_part_df.columns = ['stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']

In [34]:
CT1_weight_df = pd.concat([pg_df[['CT_1', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw']], CT1_part_df], axis=1)
CT1_weight_df.columns = ['tot_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']
CT1_weight_df = CT1_weight_df[['tot_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw', 'harvest_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw']]

CT2_weight_df = pd.concat([pg_df[['CT_2', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw']], CT2_part_df], axis=1)
CT2_weight_df.columns = ['tot_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']
CT2_weight_df = CT2_weight_df[['tot_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw', 'harvest_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw']]

CT3_weight_df = pd.concat([pg_df[['CT_3', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw']], CT3_part_df], axis=1)
CT3_weight_df.columns = ['tot_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw', 'harvest_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw']
CT3_weight_df = CT3_weight_df[['tot_fw', 'stem_fw', 'leaf_fw', 'petiole_fw', 'fruit_fw', 'harvest_fw', 'stem_dw', 'leaf_dw', 'petiole_dw', 'fruit_dw', 'harvest_dw']]

In [35]:
SW2_df = pd.read_csv('../ch2_weighing/results/2020_S/SW2_greenhouse.csv', index_col='Unnamed: 0')
SW2_df.index = pd.DatetimeIndex(SW2_df.index)

SW2_df = SW2_df.interpolate()
SW2_df = SW2_df.loc['2020-03-11 00:00:00': '2020-07-03 23:59:00']

In [36]:
env1_df = pd.concat([SW2_df['loadcell_1'], SW2_df.loc[:, 'temp':'permit']], axis=1)
env2_df = pd.concat([SW2_df['loadcell_2'], SW2_df.loc[:, 'temp':'permit']], axis=1)
env3_df = pd.concat([SW2_df['loadcell_3'], SW2_df.loc[:, 'temp':'permit']], axis=1)
env1_df.columns = [['system_w', 'temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit']]
env2_df.columns = [['system_w', 'temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit']]
env3_df.columns = [['system_w', 'temp', 'hum', 'rad', 'subs_VWC', 'subs_EC', 'subs_bulk_EC', 'subs_temp', 'permit']]

In [37]:
env_df = pd.concat([env1_df, env2_df, env3_df])

In [38]:
weight_df = pd.concat([CT1_weight_df, CT2_weight_df, CT3_weight_df])

In [39]:
INPUT_MAXS = env_df.max()
INPUT_MINS = env_df.min()
OUTPUT_MAXS = weight_df.max()
OUTPUT_MINS = weight_df.min()

In [40]:
env_df = (env_df - INPUT_MINS)/(INPUT_MAXS - INPUT_MINS)
weight_df = (weight_df - OUTPUT_MINS)/(OUTPUT_MAXS - OUTPUT_MINS)

In [41]:
env_data = env_df.values.reshape(-1, 1440, 9)
weight_data = weight_df.values

# 나중에 수정해야 함.

In [42]:
f = open('./results/2020_S/modeling_dataset.npz', 'wb')
np.savez(f,
         output_columns = weight_df.columns,
         input_indices = env_df.index,
         output_indices = weight_df.index,
         input_data = env_data,
         output_label = weight_data,
         INPUT_MAXS = INPUT_MAXS.values,
         INPUT_MINS = INPUT_MINS.values,
         OUTPUT_MAXS = OUTPUT_MAXS.values,
         OUTPUT_MINS = OUTPUT_MINS.values
        )
f.close()

# 2020 Winter

## Plant growth factors

In [ ]:
pg_df = []

In [ ]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_W/node_length.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_len'])

pg_df.append(temp_df)

In [ ]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_W/node_diameter.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_dia'])

pg_df.append(temp_df)

In [ ]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_W/plant_height.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['plant_h'])

pg_df.append(temp_df)

In [ ]:
temp_df = pd.read_csv('../ch2_weighing/results/2020_W/leaf_area.csv', index_col='Unnamed: 0')/10000
temp_index = temp_df.index
temp_df.index = pd.DatetimeIndex(temp_df.index)
pg_df.append(temp_df)

In [ ]:
pg_df = pd.concat(pg_df, axis=1)
pg_df['DAT'] = [15]*3 + [50]*6 + [80]*6 + [108]*3 + [153]*15
pg_df['date'] = pg_df.index
pg_df = pg_df.set_index(['DAT'])

In [ ]:
pg_reg_df = []
plot_x = [_ for _ in range(10, 155)]
date_x = pd.date_range('2020-09-04', '2021-01-26', freq='1d')

In [ ]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [ ]:
x = pg_df.index
y = pg_df['node_len']*10
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/10)

In [ ]:
x = pg_df.index
y = pg_df['node_dia']*10
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/10)

In [ ]:
x = pg_df.index
y = pg_df['plant_h']
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y)

In [ ]:
x = pg_df.index
y = pg_df['leaf_area']*1000
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/1000)

In [ ]:
pg_reg_df = pd.DataFrame(pg_reg_df).T
pg_reg_df.columns = ['node_len', 'node_dia', 'plant_h', 'leaf_area']
pg_reg_df.index = date_x

## Fresh weight (device)

In [ ]:
SW2_df = pd.read_csv('./results/2020_W/SW_CT_greenhouse.csv', index_col='Unnamed: 0')
SW2_df.index = pd.DatetimeIndex(SW2_df.index)

SW2_df = SW2_df.loc['2020-08-26 00:00:00': '2021-01-24 23:59:00']
SW2_df = SW2_df.interpolate()

In [ ]:
rockwool_slab = np.array([626, 708, 650, 642])
rockwool_cube = np.array([48, 46, 50, 48, 46])
rockwool = (rockwool_slab + rockwool_cube.mean()*3)/1000
rockwool_mean = rockwool.mean()

In [ ]:
substrate_volume = (120*12*7.5 + 10*10*6.5*3)/1000
water_w_df = substrate_volume*SW2_df['subs_VWC']/100
SW2_df['water'] = water_w_df

In [ ]:
SW2_df.loc[:, 'loadcell_1'] = SW2_df.loc[:, 'loadcell_1'] - 0.8
SW2_df.loc[:, 'loadcell_2'] = SW2_df.loc[:, 'loadcell_2'] - 0.5
SW2_df.loc[:, 'loadcell_3'] = SW2_df.loc[:, 'loadcell_3'] - 0.8

In [ ]:
weight_df = pd.read_csv('./results/2020_W/ct_weight.csv', index_col='Unnamed: 0')
weight_df.index = pd.DatetimeIndex(weight_df.index)
wweight_df = weight_df[['Stem FW', 'Leaf FW', 'petiole FW', 'Idv fruit FW']].sum(axis=1)

In [ ]:
roots_DW = np.array([301.25, 293.55, 315.15, 356.45, 341.55, 267.65, 303.55, 231.05, 253.05, 272.35, 334.35])/1000
roots_DW_mean = roots_DW.mean()
rs_ratio_df = pd.DataFrame([0.23255813953488372, 0.14789272030651343, 0.11954022988505748, 0.13678160919540233, 0.2835814088817321], index=weight_df.index.unique(), columns=['RS_ratio'])
DW_sum_df = weight_df[[col for col in weight_df.columns if col.endswith('DW')]].sum(axis=1).groupby(weight_df.index).mean()

In [ ]:
roots_df = (DW_sum_df.T * rs_ratio_df.T).T
roots_df.columns = ['root DW']
roots_df['root FW'] = roots_df['root DW']/0.1325
roots_df.index = pd.DatetimeIndex(roots_df.index)
wweight_wr_df = wweight_df.add(roots_df['root FW'])

In [ ]:
night_df = SW2_df.loc[SW2_df['rad'] <= 0.2, 'loadcell_1':'loadcell_3']

In [ ]:
device_df = pd.concat([(night_df.resample('1d').mean()['loadcell_1'] - SW2_df['water'].resample('1d').mean())/3-0.5,
                       (night_df.resample('1d').mean()['loadcell_2'] - SW2_df['water'].resample('1d').mean())/3-0.5,
                       (night_df.resample('1d').mean()['loadcell_3'] - SW2_df['water'].resample('1d').mean())/3-0.3], axis=1)
device_df.columns = ['CT_1', 'CT_2', 'CT_3']

In [ ]:
pg_reg_df.head()

In [ ]:
device_df.head()